In [56]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
%reload_ext autoreload

In [58]:
import pandas as pd
import seaborn as sns

In [59]:
speeches2 = pd.read_csv('millercenter_speeches_df.csv', index_col=0).reset_index(drop=True)

In [60]:
# Why the first representation of the string is not escaped and the second one is?

In [61]:
# No need to convert accented characters?

In [62]:
import sys
sys.path.append('/Users/susan_jose/Jose/springboard/nlp_political_speeches/Text_Pre-processing/')
from text_preprocessing_helpers import expand_contractions, remove_special_characters

In [63]:
# Expand contractions using DJ's function
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : expand_contractions(x))

In [64]:
# Remove special characters with DJ's function
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : remove_special_characters(x))

In [65]:
# Using SpaCy to lemmatize transcripts
# Remove stop-words

In [66]:
# No need to do POS, dependency parsing, NER or other pre-processing

In [67]:
import spacy
print(spacy.__file__)
print(spacy.__version__)

/opt/anaconda3/envs/nlp_capstone/lib/python3.7/site-packages/spacy/__init__.py
2.0.16


In [68]:
nlp = spacy.load('en')

In [69]:
def lemmatize_speech(speech):
    text = nlp(speech)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text


In [70]:
def remove_stopwords(speech):
    text = nlp(speech)
    text = ' '.join([word.text for word in text if not word.is_stop ])
    return text

In [72]:
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : remove_stopwords(x))

In [71]:
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : lemmatize_speech(x))

In [73]:
speeches.head()

,Transcript,Summary,President,Source,Date,URL,Word Count
0,34th time I speak oval office We year soon tim...,"In this broadcast from the Oval Office, Presid...",Ronald Reagan,National Archives,"January 11, 1989",https://millercenter.org/the-presidency/presid...,3289
1,thank warm welcome governor balile congressman...,NaN,Ronald Reagan,NaN,"December 16, 1988",https://millercenter.org/the-presidency/presid...,7127
2,thank thank seat live today remember know free...,"At a Veteran's Day Ceremony, Reagan praises th...",Ronald Reagan,Ronald Reagan Presidential Library,"November 11, 1988",https://millercenter.org/the-presidency/presid...,280
3,mr president mr general secretary distinguish ...,NaN,Ronald Reagan,Ronald Reagan Presidential Library,"September 26, 1988",https://millercenter.org/the-presidency/presid...,4889
4,madam chairman delegate convention fellow citi...,NaN,Ronald Reagan,NaN,"August 15, 1988",https://millercenter.org/the-presidency/presid...,4476


In [75]:
# Convert to lowercase
speeches['Transcript'] = speeches['Transcript'].str.lower()

In [76]:
# Replace newline chars by spaces
speeches['Transcript'] = speeches['Transcript'].str.replace('\n', ' ')
# Replace more than one space with just one
speeches['Transcript'] = speeches['Transcript'].str.replace('[ ]{2,}', ' ')

In [77]:
speeches['Transcript'][0]

'34th time i speak oval office we year soon time i i want share think i save long time it honor life president write past week thank i nancy i grateful opportunity serve thing presidency somewhat apart you spend lot time fast car drive people tint glassthe parent hold child wave late return time i want stop reach glass connect maybe i little tonight people ask i feel leave fact sweet sorrow sweet california ranch freedom sorrowthe goodbye course leave beautiful place you know hall stair office white house president family live favorite window i i like stand look early morning view ground washington monument mall jefferson memorial morning humidity low past jefferson river potomac virginia shore view lincoln smoke rise battle bull run i prosaic thing grass bank morning traffic people way work sailboat river i think bit window i reflect past year mean mean image come mind like refrain nautical onea small story big ship refugee sailor it early 80 height boat people sailor hard work carrie

In [79]:
def remove_non_ascii(speech):
    text = nlp(speech)
    text = ' '.join([word.text for word in text if word.is_ascii ])
    return text

In [80]:
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : remove_non_ascii(x))

In [82]:
with open('democratic_politicians.txt', 'r') as f:
    dem = f.read().split('\n')
with open('republican_politicians.txt', 'r') as g:
    rep = g.read().split('\n')

In [83]:
# Create list with affiliation per row
aff = ['Republican' if p in rep else 'Democrat' if p in dem else 'Missing' for p in speeches['President']]
#Assign affiliation to rows
speeches['Affiliation'] = aff

In [96]:
speeches.loc[speeches['Affiliation'] == 'Democrat', 'Label'] = int(1)
speeches.loc[speeches['Affiliation'] == 'Republican', 'Label'] = int(0)
speeches['Label'] = speeches['Label'].astype(int)

In [97]:
speeches.sample(10)

,Transcript,Summary,President,Source,Date,URL,Word Count,Affiliation,Label
209,president good afternoon lady gentleman march ...,President Johnson holds a press conference to ...,Lyndon B. Johnson,Lyndon Baines Johnson Library and Museum,"March 13, 1965",https://millercenter.org/the-presidency/presid...,3896,Democrat,1
176,lieutenant governor gorrell speaker cumming go...,President Johnson delivers a lengthy address a...,Lyndon B. Johnson,Lyndon Baines Johnson Library and Museum,"March 15, 1967",https://millercenter.org/the-presidency/presid...,3422,Democrat,1
57,mr president mrs ford mr vice president mr vic...,NaN,Ronald Reagan,Ronald Reagan Presidential Library,"August 19, 1976",https://millercenter.org/the-presidency/presid...,770,Republican,0
64,mr speaker mr vice president member congress f...,President Obama delivers his 2015 State of the...,Barack Obama,The White House,"January 20, 2015",https://millercenter.org/the-presidency/presid...,6858,Democrat,1
30,officer man woman corps lady gentleman i come ...,NaN,Ronald Reagan,NaN,"November 04, 1983",https://millercenter.org/the-presidency/presid...,979,Republican,0
86,ood evening tonight i report american people w...,President Obama delivers a statement on Seal T...,Barack Obama,The White House,"May 01, 2011",https://millercenter.org/the-presidency/presid...,1387,Democrat,1
363,mr speaker mr vice president member 105th cong...,In his first State of the Union address of his...,Bill Clinton,William J. Clinton Presidential Library,"February 04, 1997",https://millercenter.org/the-presidency/presid...,6746,Democrat,1
201,good evening lady gentleman i come meet leader...,NaN,Lyndon B. Johnson,National Archives,"May 02, 1965",https://millercenter.org/the-presidency/presid...,2634,Democrat,1
243,good afternoon congress pass send desk bill co...,"From the East Room of the White House, Preside...",George W. Bush,George W. Bush Presidential Library and Museum,"July 19, 2006",https://millercenter.org/the-presidency/presid...,1655,Republican,0
350,i come decision i feel i tell fellow american ...,NaN,Gerald Ford,Gerald R. Ford Presidential Library,"September 08, 1974",https://millercenter.org/the-presidency/presid...,1091,Democrat,1


In [98]:
speeches.to_csv('speeches_millercenter_preprocessed.csv', index=False)

In [84]:
speeches.head()

,Transcript,Summary,President,Source,Date,URL,Word Count,Affiliation
0,34th time i speak oval office we year soon tim...,"In this broadcast from the Oval Office, Presid...",Ronald Reagan,National Archives,"January 11, 1989",https://millercenter.org/the-presidency/presid...,3289,Republican
1,thank warm welcome governor balile congressman...,NaN,Ronald Reagan,NaN,"December 16, 1988",https://millercenter.org/the-presidency/presid...,7127,Republican
2,thank thank seat live today remember know free...,"At a Veteran's Day Ceremony, Reagan praises th...",Ronald Reagan,Ronald Reagan Presidential Library,"November 11, 1988",https://millercenter.org/the-presidency/presid...,280,Republican
3,mr president mr general secretary distinguish ...,NaN,Ronald Reagan,Ronald Reagan Presidential Library,"September 26, 1988",https://millercenter.org/the-presidency/presid...,4889,Republican
4,madam chairman delegate convention fellow citi...,NaN,Ronald Reagan,NaN,"August 15, 1988",https://millercenter.org/the-presidency/presid...,4476,Republican
